##### Name: Usama Khalid
##### Roll: i19-1236

In [1]:
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

### 1. Read and Visualize the data

In [2]:
wine_df = pd.read_csv("winequality-red.csv",sep=';')
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### 2. Dataframe to Torch Dataset

In [3]:
X = torch.from_numpy(wine_df.iloc[:,:-1].to_numpy()).type(torch.float)
y = torch.from_numpy(wine_df.iloc[:,-1].to_numpy()).type(torch.float)

wine_ds = TensorDataset(X, y)

### 3. Dataset Train Test Split

In [4]:
random_seed = 45
torch.manual_seed(random_seed)

test_pct = 0.2
val_size = int(len(wine_ds)*test_pct)
train_size = len(wine_ds) - val_size

train_ds, val_ds = torch.utils.data.random_split(wine_ds, [train_size, val_size])
len(train_ds), len(val_ds)

(1280, 319)

### 4. Dataloader Train Test Split

In [5]:
batch_size=50
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size)

In [16]:
def print_stats( epoch, num_epochs, result):
    if (epoch+1) % 100 == 0 or epoch == num_epochs-1:
        print("[{}] validation loss: {:.4f}".format(epoch+1, result))


class WineModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(11,9)
        self.linear2 = nn.Linear(9,6)
        self.linear3 = nn.Linear(6,3)
        self.bn = nn.BatchNorm1d(3)
        self.linear4 = nn.Linear(3,1)


    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = self.bn(x)
        x = self.linear4(x)
        return x

    def fit_evaluate(self,num_epochs,learning_rate,train_dataloader,validation_dataloader,optimizer_func=torch.optim.Adam):
        result = []
        optimizer = optimizer_func(self.parameters(),lr=learning_rate)
        for epoch in range(num_epochs):
            for batch in train_dataloader:
                loss = self.step(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            result.append(self.evaluate(validation_dataloader))
            print_stats(epoch,num_epochs,result[-1])
        return result


    def evaluate(self, dataloader):
        losses = [self.step(batch).detach() for batch in dataloader]
        epoch_loss = torch.stack(losses).mean()
        return epoch_loss.item()

    def step(self, batch):
        inputs, label = batch
        label = label.view(-1,1)
        output = self(inputs)
        loss = F.l1_loss(output,label)
        return loss


In [20]:
model=WineModel()
epochs = 5000
lr = 1e-5
results = model.fit_evaluate(epochs, lr, train_dl,val_dl)



[100] validation loss: 5.1326
[200] validation loss: 5.0756
[300] validation loss: 5.0140
[400] validation loss: 4.9481
[500] validation loss: 4.8778
[600] validation loss: 4.8034
[700] validation loss: 4.7247
[800] validation loss: 4.6418
[900] validation loss: 4.5548
[1000] validation loss: 4.4635
[1100] validation loss: 4.3681
[1200] validation loss: 4.2686
[1300] validation loss: 4.1649
[1400] validation loss: 4.0570
[1500] validation loss: 3.9451
[1600] validation loss: 3.8290
[1700] validation loss: 3.7087
[1800] validation loss: 3.5843
[1900] validation loss: 3.4559
[2000] validation loss: 3.3232
[2100] validation loss: 3.1864
[2200] validation loss: 3.0454
[2300] validation loss: 2.9004
[2400] validation loss: 2.7513
[2500] validation loss: 2.5999
[2600] validation loss: 2.4451
[2700] validation loss: 2.2860
[2800] validation loss: 2.1224
[2900] validation loss: 1.9552
[3000] validation loss: 1.7848
[3100] validation loss: 1.6133
[3200] validation loss: 1.4373
[3300] validation